In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Nvidia-Stock-Price-All-Time'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data

            Date        Open        High         Low       Close   Adj Close  \
0     1999-01-22    0.437500    0.488281    0.388021    0.410156    0.376820   
1     1999-01-25    0.442708    0.458333    0.410156    0.453125    0.416296   
2     1999-01-26    0.458333    0.467448    0.411458    0.417969    0.383998   
3     1999-01-27    0.419271    0.429688    0.395833    0.416667    0.382801   
4     1999-01-28    0.416667    0.419271    0.412760    0.415365    0.381605   
...          ...         ...         ...         ...         ...         ...   
5700  2021-09-16  221.830002  222.770004  219.270004  222.419998  222.419998   
5701  2021-09-17  223.000000  223.210007  218.300003  219.000000  219.000000   
5702  2021-09-20  211.460007  214.330002  206.619995  211.130005  211.130005   
5703  2021-09-21  214.169998  214.250000  209.500000  212.460007  212.460007   
5704  2021-09-22  213.660004  219.600006  211.960007  219.410004  219.410004   

         Volume  
0     271468800  
1  

In [4]:
data = torch.from_numpy(np.array(data)).to(device).float().view(1,-1)

In [5]:
data = data['Open'].astype(float).tolist()

In [6]:
data = torch.from_numpy(np.array(data)).to(device).float().view(1,-1)

In [7]:
data_input = data[:1,:-1]

In [8]:
data_target = data[:1,1:]

In [9]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 512
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1 = torch.zeros(batch_size,self.hidden).to(device)
        c_t1 = torch.zeros(batch_size,self.hidden).to(device)
        h_t2 = torch.zeros(batch_size,self.hidden).to(device)
        c_t2 = torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [10]:
model = Model().to(device)
crtierion = MSELoss()

In [11]:
optimizer = Adam(model.parameters(),lr=0.001)

In [12]:
epochs = 100

In [13]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = crtierion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        n = data_input.shape[1]
        future = 100
        preds = model(data_input,future)
        loss = crtierion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].view(-1).cpu().detach().numpy()
        plt.figure(figsize=(12,7))
        plt.plot(np.arange(n),data_target.view(-1).cpu().detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()